In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.listdir()
os.chdir("drive/MyDrive/Purdue-Second Year/Fall Semester/CS 50100")

In [84]:
import torch
import csv
import numpy as np
# from lstmunivariate_training import LSTMModel
import lstmunivariate_training as lstm

In [85]:
# Initialize the LSTM model
input_size = 1
hidden_size = 64
num_layers = 1
lstm_model = lstm.LSTMModel(input_size, hidden_size, num_layers) # Replace with model parameters

# Load the saved model's state dictionary
lstm_model.load_state_dict(torch.load('lstm_model.pth'))

# Set model to evaluation mode
lstm_model.eval()

LSTMModel(
  (lstm): LSTM(1, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [86]:
# Evaluate with 2021 Data
unrate = [] # Initialize list for univariate unemployment rate
eval_dict = {} # Initialize a dictionary for evaluation
actual_values = {} # Dictionary for storing actual 2021 values
# Open the .csv and extract unemployment data
with open ('macro_monthly_clean.csv', newline = '') as c:
  reader = csv.DictReader(c)
  # Since we will be predicting data based on the previous year's (same month), sort data into a dictionary based on months
  for row in reader:
    date_ls = row['DATE'].split('-')
    year_interest = int(date_ls[0])
    if (year_interest >= 2012 and year_interest != 2021): # Include only entries from 2012 for test
      if date_ls[1] not in eval_dict.keys():
        eval_dict[date_ls[1]] = []
      eval_dict[date_ls[1]].append(float(row['unrate']))
    elif year_interest == 2021:
      if date_ls[1] not in actual_values.keys():
        actual_values[date_ls[1]] = float(row['unrate'])
months = eval_dict.keys()
if len(months) != len(actual_values.values()):
  print('Months do not match up with prediction values within actual values')
  print('Months: ' + str(len(months)))
  print('Actual values for test: ' + str(len(actual_values.values())))


Months do not match up with prediction values within actual values
Months: 12
Actual values for test: 10


In [87]:
# Initialize list for predictions
predictions_ls = []
rmse_avg = []  # Initialize list for root mean square average
mae_avg = []   # Initialize list for mean absolute error average
mape_avg = []  # Initialize list for mean absolute percentage error average

# Make predictions using the loaded model
with torch.no_grad():  # Disable gradient tracking for inference
    for key in eval_dict.keys():
        # Since we do not have months November & December in 2021, remove their predictions
        if key != '11' and key != '12':
            # Convert input data for 2021 to a PyTorch tensor (similar to what you did for training data)
            input_data_tensor = torch.tensor(eval_dict[key], dtype=torch.float32)
            # Reshape the input data to match the expected format (batch_size, sequence_length, input_size)
            input_data_tensor = input_data_tensor.view(1, -1, 1)  # Reshape to (1, sequence_length, 1)

            predictions = lstm_model(input_data_tensor).tolist()[0]  # predictions here is still a list
            # print(f'Prediction for unemployment rate month {key}: {predictions}')
            predictions_ls.append(predictions[0])

            # Calculate Root mean squared error
            mse = np.mean((predictions[0] - actual_values[key]) ** 2)
            rmse = np.sqrt(mse)
            rmse_avg.append(rmse)

            # Calculate Mean Absolute Error
            mae = np.mean(np.abs(predictions[0] - actual_values[key]))
            mae_avg.append(mae)

            # Calculate Mean Absolute Percentage Error
            mape = np.mean(np.abs((predictions[0] - actual_values[key]) / actual_values[key])) * 100
            mape_avg.append(mape)

# Calculate average mean squared error
average_rmse = sum(rmse_avg) / len(rmse_avg)
average_mae = sum(mae_avg) / len(mae_avg)
average_mape = sum(mape_avg) / len(mape_avg)

print(f"Average Root Mean Square Error for prediction: {average_rmse:.2f}")
print(f"Average Mean Absolute Error for prediction: {average_mae:.2f}")
print(f"Average Mean Absolute Percentage Error for prediction: {average_mape:.2f}%")


Average Root Mean Square Error for prediction: 0.60
Average Mean Absolute Error for prediction: 0.60
Average Mean Absolute Percentage Error for prediction: 11.74%
